# Shopee Code League - Multi-Channel Contacts
Data Analytics Challenge
March, 2021

Contributors of this code:
- [LiaoWC](https://github.com/LiaoWC)
- [Ch1sInTheHouse](https://github.com/Chr1sInTheHouse)

Kaggle page: https://www.kaggle.com/c/scl-2021-da/overview

### Method
We know that all tickets are unique, and that any two people have the same phone, email, or order ID are exactly the same people.
Let's view all tickets as different nodes(vertices).
We check phone, email, and order ID. If any two ticket are belong to the same person, we construct a
undirected edge between them. Finally, we find the graph's connected components.
Each unique user has a corresponding connected components.


### Time complexity

Assume number of total tickets is `n`.

Sorting takes `O(nlogn)` time.<br>
Getting edges by iterating takes `O(n)` time.<br>
Finding connected component takes `o(n^2)` time[1].<br>
Grouping takes `O(nlogn)` time.<br>
Finding group data and write file takes `O(nlogn)` time.<br>

The total time complexity is `o(n^2)`.

---
[1] Common ways of finding connected components like Tarjan's algorithm
using adjacent list takes not more than `O(number of vertices + number of edges)`.
In this case, we use `scipy`'s connected component function. We've tested that it seems not take
up to n^2 time and it's fast. If you're interested in the algorithm, you can find more information about how `scipy`
implements this function.


In [16]:
# Import packages
import scipy
from scipy import sparse
from scipy.sparse.csgraph import connected_components
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np

In [17]:
# Load data
df = pd.read_json('./contacts.json')

### Find out all adjacent edges
Check three columns: email, phone, and order ID respectively.
For each column, we sort first, and then iterate one time to get edges.
Note that for every connected component, we just need at least one path that any nodes can go to any other nodes.
That is, if node 1, 2, 3, 4 are connected component,
we just need [1 -> 2], [1 -> 3], [1 -> 4] to show they are belong to the same connected component.
##### Example
If node 1, 2, 3, 4 have the same email, after sorting it by email,
they will be arranged like this:
```
node 1
node 2
node 3
node 4
```
and we will create edges: [1 -> 2], [1 -> 3], [1 -> 4] in the next iterating phase.


In [18]:
# Sort
Email = df.sort_values(by=['Email']).to_numpy()[:, (0, 1)]
Phone = df.sort_values(by=['Phone']).to_numpy()[:, (0, 2)]
OrderId = df.sort_values(by=['OrderId']).to_numpy()[:, (0, 4)]

# Get edges by observing email
gr = []
now_mail = Email[0]
for i in range(1, 500000):
    if Email[i][1] == now_mail[1] and now_mail[1] !='':
        gr.append([Email[i][0], now_mail[0]])
    else:
        now_mail = Email[i]

# Get edges by observing phone
now_mail = Phone[0]
for i in range(1, 500000):
    if Phone[i][1] == now_mail[1] and now_mail[1] !='':
        gr.append([Phone[i][0], now_mail[0]])
    else:
        now_mail = Phone[i]

# Get edges by observing order ID
now_mail = OrderId[0]
for i in range(1, 500000):
    if OrderId[i][1] == now_mail[1] and now_mail[1] !='':
        gr.append([OrderId[i][0], now_mail[0]])
    else:
        now_mail = OrderId[i]

### Get sparse matrix that represent
We get sparse matrix using the method from the reference:
https://stackoverflow.com/questions/38665388/how-to-read-in-an-edge-list-to-make-a-scipy-sparse-matrix

In [19]:
# Add self-connected edge to make matrix be square.
for i in range(500000):
    gr.append([i, i])


In [20]:
# For conveniently getting ticket info
def get_ticket_data_by_id(id_num: int):
    return df.loc[df['Id'] == id_num]

In [21]:
# Give all edges and get csr matrix and number of components and labels
def to_m(edges):
    arr = np.array(edges)
    k1, k2, k3=np.unique(arr,return_inverse=True,return_index=True)
    rows, cols=k3.reshape(arr.shape).T
    mat = sparse.coo_matrix((np.ones(rows.shape,int),(rows,cols)))
    csr_mat = mat.tocsr()
    # return csr_mat
    n_components, labels = connected_components(csgraph=csr_mat, directed=False, return_labels=True)
    return csr_mat, n_components, labels

m, n_c, lab = to_m(gr)
# Use "m.A" to show
# m: sparse matrix
# n_c: number of different people
# lab: array which shows every ticket (idx is ticket_id)'s group_id. (groups means people here)

### Grouping

In [22]:
# Binding with ticket_id and sorting it allow us to group in O(n)
lab_idx_and_ticket_id = []
for i, g_id in enumerate(lab):
    lab_idx_and_ticket_id.append([g_id, i])

lab_idx_and_ticket_id.sort()

In [23]:
# Grouping (group info of tickets that belong to the same user)
# It takes lots of time.

max_group_idx = max(lab)

# group_idx -> {'ticket_ids': [...], 'ticket_trace': str, 'contact': int}
all_groups = {}

cur_idx = 0
for group_idx in range(0,max_group_idx + 1):
    group = {}
    cur_ticket_trace = ''
    cur_contact_sum = 0
    ticket_ids = []
    if cur_idx % 10000 == 0:
        # Print info to know the current progress
        print('group: {}; from {}'.format(group_idx, cur_idx))
    for idx in range(cur_idx, len(lab)):
        if lab_idx_and_ticket_id[idx][0] == group_idx:
            ticket_id = lab_idx_and_ticket_id[idx][1]
            ticket_data = get_ticket_data_by_id(ticket_id)
            cur_ticket_trace += '-' + str(ticket_id)
            cur_contact_sum += ticket_data['Contacts'].tolist()[0]
            ticket_ids.append(ticket_id)
        cur_idx += 1
        if cur_idx >= 500000 or lab_idx_and_ticket_id[cur_idx][0] != group_idx:
            break
    group['ticket_ids'] = ticket_ids
    group['ticket_trace'] = cur_ticket_trace[1:] # Remove first dash
    group['contact'] = cur_contact_sum
    all_groups[group_idx] = group

group: 0; from 0
group: 11679; from 40000
group: 33061; from 100000
group: 41139; from 120000
group: 49719; from 140000
group: 58625; from 160000
group: 82779; from 210000
group: 87941; from 220000
group: 109776; from 260000
group: 115583; from 270000
group: 127485; from 290000
group: 133734; from 300000
group: 140133; from 310000
group: 146645; from 320000
group: 153241; from 330000
group: 159980; from 340000
group: 173935; from 360000
group: 181142; from 370000
group: 188443; from 380000
group: 195897; from 390000
group: 203675; from 400000
group: 211574; from 410000
group: 219663; from 420000
group: 227943; from 430000
group: 236400; from 440000
group: 245105; from 450000
group: 254012; from 460000
group: 263120; from 470000
group: 272453; from 480000
group: 282049; from 490000


### Get answers and write csv file

In [24]:
# Sorting it to make us take only O(n) to get things by iterating it in O(n)
arr = sorted(lab_idx_and_ticket_id,key=lambda x:x[1])

In [25]:
# Save answers in csv file

# Type filename
file_name = input()
with open('{}.csv'.format(file_name), 'w+') as file:
    writer = csv.writer(file)
    # Write column info
    writer.writerow(['ticket_id','ticket_trace/contact'])
    # Iterate to get group id to get group's data and write into the csv file
    for ticket_id in range(0,len(lab)):
        try:
            group_id = arr[ticket_id][0]
            group = all_groups[group_id]
            # if ticket_id in group['ticket_ids']:
            writer.writerow([ticket_id, group['ticket_trace']+', '+str(group['contact'])])
        except:
            pass